# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [75]:
NUM_VIDEOS = 10
GRID_SIZE = 2
BINS = [int(180/1), int(256/1)]
HIST_FRAME_SKIP = 1
REFRESH = False

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [76]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 10 - Grid 2 - Bins [180, 256] - Skip 1


In [77]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = prep.get_test_video_set(NUM_VIDEOS, GRID_SIZE, BINS, n=10)

# test_set, labels = generate_test_segments(training_set, n=100, duration=20)

processing 10/10

In [78]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:    {:d}".format( len(training_set)) )
print("Num. segments:  {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:       {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:     {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms: {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("\nTEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:    10
Num. segments:  972
Duration:       3,053.0 s
Num frames:     82526
Num histograms: 82526

TEST SET:
Size: 10


# Small manual test

In [10]:
pr = False
for i in range(0,1):
    for j in range(5):
        x = random.choice(range(len(test_set[i])))
        found = search.findFrame(test_set[i][x], training_set, cv2.HISTCMP_CHISQR, 5, hist_frame_skip=HIST_FRAME_SKIP, prints= pr, warnings=pr)
        print('Found {} - Expected {}'.format(found, labels[i]))

        
# test_histograms = prep.get_test_video("{:05d}".format(5), GRID_SIZE, BINS)
# for i in range(10):
#     found = search.findFrame(test_histograms[i], training_set, cv2.HISTCMP_CHISQR, 10, \
#                              hist_frame_skip=HIST_FRAME_SKIP, prints= pr, warnings=pr)
#     print(found)

Found ('00002.mp4', 1188) - Expected ('00002.mp4', 1111, 666)
Found ('00002.mp4', 1051) - Expected ('00002.mp4', 1111, 666)
Found ('00002.mp4', 1049) - Expected ('00002.mp4', 1111, 666)
Found ('00002.mp4', 1189) - Expected ('00002.mp4', 1111, 666)
Found ('00002.mp4', 1049) - Expected ('00002.mp4', 1111, 666)


## Run model on test set

In [9]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
# for method in [cv2.HISTCMP_BHATTACHARYYA]:
    %timeit -n 1 search.findFrame_old(test_set[0][0], training_set, method, warnings = False)
    %timeit -n 1 search.findFrame(test_set[0][0], training_set, method, warnings = False)
    print()

# for ch in [[0], [1], [0, 1]]:
#     print('{}'.format(ch))
#     %timeit -n 10 search.findFrame(test_set[0], training_set, cv2.HISTCMP_CORREL, channels=ch)

969 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
822 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.12 s ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
944 ms ± 6.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

925 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
786 ms ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.06 s ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
876 ms ± 9.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.25 s ± 44.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.06 s ± 9.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.71 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.53 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



In [79]:
results = []

for i, histogram in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(histogram)), end='', flush=True)
    
    results.append(search.findFrame(histogram[0], training_set, cv2.HISTCMP_CHISQR_ALT, 2, \
                                    hist_frame_skip=HIST_FRAME_SKIP, warnings = False))


Searching segment 10/10

## Evaluate performance

In [80]:
movie_results, start_frame_dist = evaluate(results, labels)

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, \
             movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS\n")

printParams()
print("\nCorrect video: {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment: {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to start frame: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(
    start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/25))

TEST RESULTS

Num. Vid 10 - Grid 2 - Bins [180, 256] - Skip 1

Correct video: 10 / 10 (100.0%)
Inside fragment: 9 / 10 (90.0%)
Average distance to start frame: 136 +/- 197 frames (approx. 5.4 sec)
